- Configurações iniciais, imports e definição da pasta de trabalho

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive montado com sucesso.")
except ModuleNotFoundError:
    print("AVISO: Não está no ambiente Google Colab ou erro ao montar Drive.")
except Exception as e:
    print(f"Erro ao montar o Google Drive: {e}")

DRIVE_BASE_PATH_S3 = "/content/drive/MyDrive/Colab Notebooks/FIAP/Global"

if not os.path.isdir(DRIVE_BASE_PATH_S3):
  print(f"AVISO: Diretório base '{DRIVE_BASE_PATH_S3}' não encontrado no Drive. Verifique os caminhos.")

CAMINHO_ARQUIVO_CSV = os.path.join(DRIVE_BASE_PATH_S3, "dados-inmet-sp.csv")

CAMINHO_DA_PASTA_DE_IMAGENS = os.path.join(DRIVE_BASE_PATH_S3, "images")

LISTA_NOMES_IMAGENS = []
EXTENSOES_IMAGEM_VALIDAS = ('.jpg', '.jpeg', '.png')

print(f"Tentando ler imagens da pasta: {CAMINHO_DA_PASTA_DE_IMAGENS}")
if os.path.exists(CAMINHO_DA_PASTA_DE_IMAGENS) and os.path.isdir(CAMINHO_DA_PASTA_DE_IMAGENS):
    try:
        for nome_arquivo_na_pasta in os.listdir(CAMINHO_DA_PASTA_DE_IMAGENS):
            if nome_arquivo_na_pasta.lower().endswith(EXTENSOES_IMAGEM_VALIDAS):
                LISTA_NOMES_IMAGENS.append(nome_arquivo_na_pasta)

        if LISTA_NOMES_IMAGENS:
            print(f"{len(LISTA_NOMES_IMAGENS)} imagens encontradas na pasta.")
            print("Amostra de nomes de imagens carregados:", LISTA_NOMES_IMAGENS[:5]) # Mostra os 5 primeiros
        else:
            print("Nenhuma imagem encontrada com as extensões válidas na pasta especificada.")
            print("Verifique o caminho da pasta e se as imagens têm extensões como .jpg, .jpeg, ou .png.")

    except Exception as e:
        print(f"Erro ao listar arquivos da pasta '{CAMINHO_DA_PASTA_DE_IMAGENS}': {e}")
        LISTA_NOMES_IMAGENS = [] # Garante lista vazia em caso de erro na leitura
else:
    print(f"Erro: A pasta de imagens '{CAMINHO_DA_PASTA_DE_IMAGENS}' não foi encontrada ou não é um diretório válido. Verifique o caminho.")
    LISTA_NOMES_IMAGENS = [] # Garante lista vazia

# Definindo os limiares de alerta
MEUS_LIMIARES_DE_ALERTA = [
    {'coluna_acumulado': 'prec_acum_1h', 'valor_mm': 25, 'nivel_alerta': 'ALERTA MÁXIMO', 'mensagem': 'Chuva MUITO FORTE em 1h (>25mm)'},
    {'coluna_acumulado': 'prec_acum_1h', 'valor_mm': 15, 'nivel_alerta': 'ALERTA', 'mensagem': 'Chuva FORTE em 1h (>15mm)'},
    {'coluna_acumulado': 'prec_acum_24h', 'valor_mm': 100, 'nivel_alerta': 'ALERTA MÁXIMO', 'mensagem': 'Acumulado >100mm em 24h - RISCO ALTO'},
    {'coluna_acumulado': 'prec_acum_24h', 'valor_mm': 70, 'nivel_alerta': 'ALERTA', 'mensagem': 'Acumulado >70mm em 24h - RISCO MODERADO'},
    {'coluna_acumulado': 'prec_acum_24h', 'valor_mm': 40, 'nivel_alerta': 'ATENÇÃO', 'mensagem': 'Acumulado >40mm em 24h - Atenção'},
    {'coluna_acumulado': 'prec_acum_72h', 'valor_mm': 150, 'nivel_alerta': 'ALERTA', 'mensagem': 'Acumulado >150mm em 72h (solo saturado)'},
]

In [ ]:
def carregar_dados_climaticos(
    caminho_arquivo_csv,
    nome_coluna_data_completa='data',
    nome_coluna_hora_completa='hora',
    nome_coluna_precipitacao='precipitacao_total',
    nome_coluna_estacao='estacao',
    nome_coluna_latitude='latitude',
    nome_coluna_longitude='longitude',
    nome_coluna_municipio='municipio',
    nome_coluna_estado='estado',
    linhas_cabecalho=0,
    separador=',',
    codificacao='utf-8',
    formato_decimal='.'
):
    """
    Carrega e pré-processa dados climáticos de um arquivo CSV.
    """
    try:
        df = pd.read_csv(
            caminho_arquivo_csv,
            skiprows=linhas_cabecalho,
            sep=separador,
            encoding=codificacao,
            decimal=formato_decimal,
            na_values=['', ' ', 'NULL', '-9999']
        )
        print(f"Arquivo '{caminho_arquivo_csv}' lido com sucesso.")

    except FileNotFoundError:
        print(f"Erro: O arquivo '{caminho_arquivo_csv}' não foi encontrado.")
        return None
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV: {e}")
        return None

    colunas_map = {
        'data_str': nome_coluna_data_completa,
        'hora_str': nome_coluna_hora_completa,
        'precipitacao': nome_coluna_precipitacao,
        'estacao': nome_coluna_estacao,
        'latitude': nome_coluna_latitude,
        'longitude': nome_coluna_longitude,
        'municipio': nome_coluna_municipio,
        'estado': nome_coluna_estado
    }

    colunas_para_selecionar_renomear = {}
    colunas_obrigatorias = ['data_str', 'hora_str', 'precipitacao', 'estacao']

    for nome_logico, nome_no_arquivo in colunas_map.items():
        if nome_no_arquivo in df.columns:
            colunas_para_selecionar_renomear[nome_no_arquivo] = nome_logico
        elif nome_logico in colunas_obrigatorias:
            print(f"Erro: A coluna obrigatória '{nome_no_arquivo}' (para {nome_logico}) não foi encontrada.")
            print(f"Colunas disponíveis: {df.columns.tolist()}")
            return None

    try:
        colunas_existentes_no_csv_para_usar = [col_csv for col_csv in colunas_para_selecionar_renomear.keys() if col_csv in df.columns]
        df_selecionado = df[colunas_existentes_no_csv_para_usar].copy()
        df_selecionado.rename(columns=colunas_para_selecionar_renomear, inplace=True)
    except KeyError as e:
        print(f"Erro ao selecionar/renomear colunas. Detalhe: {e}")
        return None

    if 'precipitacao' in df_selecionado.columns:
        df_selecionado['precipitacao'] = pd.to_numeric(df_selecionado['precipitacao'], errors='coerce')
        df_selecionado['precipitacao'] = df_selecionado['precipitacao'].fillna(0) # Corrigido
    else:
        print("Erro: Coluna 'precipitacao' não está no DataFrame selecionado após o mapeamento.")
        return None

    try:
        datetime_str = df_selecionado['data_str'].astype(str) + ' ' + df_selecionado['hora_str'].astype(str)
        df_selecionado['timestamp'] = pd.to_datetime(datetime_str, errors='coerce')
    except Exception as e:
        print(f"Erro ao converter data e hora para timestamp: {e}")
        return None

    df_selecionado.dropna(subset=['timestamp'], inplace=True)
    df_selecionado.set_index('timestamp', inplace=True)

    colunas_finais_desejadas = ['precipitacao', 'estacao', 'latitude', 'longitude', 'municipio', 'estado']
    colunas_finais_presentes = [col for col in colunas_finais_desejadas if col in df_selecionado.columns]
    df_final = df_selecionado[colunas_finais_presentes]

    print("\n--- Amostra dos dados processados ---")
    print(df_final.head())
    print("\n--- Informações dos dados processados ---")
    df_final.info()

    return df_final

In [ ]:
def calcular_precipitacao_acumulada(df, janelas_horas=[1, 3, 6, 12, 24, 72]):
    """
    Calcula a precipitação acumulada, agrupando por 'estacao'.
    """
    if not isinstance(df, pd.DataFrame):
        print("Erro: A entrada não é um DataFrame do Pandas.")
        return None
    if 'precipitacao' not in df.columns:
        print("Erro: Coluna 'precipitacao' não encontrada.")
        return None
    if 'estacao' not in df.columns:
        print("Erro: Coluna 'estacao' não encontrada. Cálculo por estação não é possível.")
        return None
    if not isinstance(df.index, pd.DatetimeIndex):
        print("Erro: O DataFrame não possui um DatetimeIndex.")
        return None

    print(f"\nCalculando precipitação acumulada para janelas: {janelas_horas} horas...")
    df_copia = df.copy()
    df_copia.sort_values(by=['estacao', df_copia.index.name], inplace=True)

    for janela_h in janelas_horas:
        nome_coluna_acumulada = f'prec_acum_{janela_h}h'
        print(f"  Calculando para {janela_h}h...")
        df_copia[nome_coluna_acumulada] = df_copia.groupby('estacao')['precipitacao'] \
                                               .rolling(window=janela_h, min_periods=1) \
                                               .sum() \
                                               .reset_index(level='estacao', drop=True)
    print("\nCálculo de precipitação acumulada concluído.")
    return df_copia

In [ ]:
def gerar_alertas_precipitacao(df_dados, limiares):
    """
    Gera alertas de precipitação com base em limiares definidos, usando 'estacao'.
    """
    if not isinstance(df_dados, pd.DataFrame) or not isinstance(df_dados.index, pd.DatetimeIndex):
        print("Erro: df_dados deve ser um DataFrame com DatetimeIndex.")
        return pd.DataFrame()
    if not isinstance(limiares, list) or not all(isinstance(item, dict) for item in limiares):
        print("Erro: limiares deve ser uma lista de dicionários.")
        return pd.DataFrame()
    if 'estacao' not in df_dados.columns:
        print("Erro: Coluna 'estacao' não encontrada no DataFrame.")
        return pd.DataFrame()

    alertas_gerados = []
    print("\nVerificando alertas...")
    colunas_extras_possiveis = ['latitude', 'longitude', 'municipio', 'estado']

    for _, linha in df_dados.iterrows():
        timestamp = linha.name
        nome_estacao = linha['estacao']
        dados_extras_estacao = {col_extra: linha.get(col_extra) for col_extra in colunas_extras_possiveis if col_extra in linha}

        for limiar in limiares:
            coluna_gatilho = limiar['coluna_acumulado']
            valor_limite = limiar['valor_mm']
            nivel = limiar['nivel_alerta']
            msg_template = limiar['mensagem']

            if coluna_gatilho in linha:
                valor_observado = linha[coluna_gatilho]
                if pd.notna(valor_observado) and valor_observado > valor_limite:
                    alerta = {
                        'timestamp': timestamp,
                        'estacao': nome_estacao,
                        'nivel_alerta': nivel,
                        'descricao_alerta': msg_template,
                        'coluna_trigger': coluna_gatilho,
                        'valor_observado_mm': valor_observado,
                        'limiar_mm': valor_limite
                    }
                    alerta.update(dados_extras_estacao)
                    alertas_gerados.append(alerta)

    if not alertas_gerados:
        print("Nenhum alerta gerado com os critérios atuais.")
        return pd.DataFrame()

    df_alertas = pd.DataFrame(alertas_gerados)
    niveis_ordem = ['ATENÇÃO', 'ALERTA', 'ALERTA MÁXIMO']
    if not df_alertas.empty and 'nivel_alerta' in df_alertas.columns:
        df_alertas['nivel_alerta'] = pd.Categorical(df_alertas['nivel_alerta'], categories=niveis_ordem, ordered=True)
        df_alertas.sort_values(by=['timestamp', 'estacao', 'nivel_alerta'], ascending=[True, True, False], inplace=True)

    print(f"{len(df_alertas)} alertas gerados.")
    return df_alertas

In [ ]:
def extrair_data_de_nome_arquivo(nome_arquivo):
    """
    Tenta extrair uma data (AAAA-MM-DD ou DD-MM-AAAA) de um nome de arquivo.
    """
    match_iso = re.search(r'(\d{4})-(\d{2})-(\d{2})', nome_arquivo)
    if match_iso:
        ano, mes, dia = map(int, match_iso.groups())
        try: return datetime(ano, mes, dia).date()
        except ValueError: pass
    match_dmY = re.search(r'(\d{2})-(\d{2})-(\d{4})', nome_arquivo)
    if match_dmY:
        dia, mes, ano = map(int, match_dmY.groups())
        try: return datetime(ano, mes, dia).date()
        except ValueError: pass
    return None

In [ ]:
def extrair_data_de_nome_arquivo(nome_arquivo):
    """
    Tenta extrair uma data (AAAA-MM-DD ou DD-MM-AAAA) de um nome de arquivo.
    """
    match_iso = re.search(r'(\d{4})-(\d{2})-(\d{2})', nome_arquivo)
    if match_iso:
        ano, mes, dia = map(int, match_iso.groups())
        try: return datetime(ano, mes, dia).date()
        except ValueError: pass
    match_dmY = re.search(r'(\d{2})-(\d{2})-(\d{4})', nome_arquivo)
    if match_dmY:
        dia, mes, ano = map(int, match_dmY.groups())
        try: return datetime(ano, mes, dia).date()
        except ValueError: pass
    return None

In [ ]:
def correlacionar_alertas_com_imagens(df_alertas, lista_nomes_imagens):
    """
    Correlaciona alertas de precipitação com imagens baseado na data, esperando 'estacao'.
    """
    if not isinstance(df_alertas, pd.DataFrame) or 'timestamp' not in df_alertas.columns:
        print("Erro: df_alertas deve ser um DataFrame com a coluna 'timestamp'.")
        return pd.DataFrame()
    if 'estacao' not in df_alertas.columns:
         print("Erro: df_alertas deve conter a coluna 'estacao'.")
         return pd.DataFrame()
    if not isinstance(lista_nomes_imagens, list):
        print("Erro: lista_nomes_imagens deve ser uma lista.")
        return pd.DataFrame()

    mapa_data_para_imagens = {}
    print("\nProcessando nomes de arquivos de imagem...")
    for nome_imagem in lista_nomes_imagens:
        data_imagem = extrair_data_de_nome_arquivo(nome_imagem)
        if data_imagem:
            print(f"  Imagem: '{nome_imagem}' -> Data: {data_imagem}")
            if data_imagem not in mapa_data_para_imagens:
                mapa_data_para_imagens[data_imagem] = []
            mapa_data_para_imagens[data_imagem].append(nome_imagem)
        else:
            print(f"  Imagem: '{nome_imagem}' -> Data não extraída.")

    if not mapa_data_para_imagens:
        print("Nenhuma data pôde ser extraída dos nomes das imagens.")
        return pd.DataFrame()

    print("\nCorrelacionando alertas com imagens...")
    alertas_correlacionados = []
    df_alertas_copia = df_alertas.copy()
    if not pd.api.types.is_datetime64_any_dtype(df_alertas_copia['timestamp']):
         df_alertas_copia['timestamp'] = pd.to_datetime(df_alertas_copia['timestamp'], errors='coerce')
    df_alertas_copia['data_alerta_obj'] = df_alertas_copia['timestamp'].dt.date

    for _, alerta_linha in df_alertas_copia.iterrows():
        data_alerta = alerta_linha['data_alerta_obj']
        if pd.isna(data_alerta): continue

        if data_alerta in mapa_data_para_imagens:
            imagens_do_dia = mapa_data_para_imagens[data_alerta]
            for nome_img_associada in imagens_do_dia:
                alerta_com_imagem = alerta_linha.to_dict()
                alerta_com_imagem['imagem_associada'] = nome_img_associada
                alertas_correlacionados.append(alerta_com_imagem)

    if not alertas_correlacionados:
        print("Nenhuma correlação encontrada entre alertas e imagens.")
        return pd.DataFrame()

    df_resultado_correlacao = pd.DataFrame(alertas_correlacionados)
    if 'data_alerta_obj' in df_resultado_correlacao.columns:
        df_resultado_correlacao.drop(columns=['data_alerta_obj'], inplace=True)

    print(f"{len(df_resultado_correlacao)} alertas foram correlacionados com imagens.")
    return df_resultado_correlacao

In [ ]:
# 1. Carregar os dados
dados_base = carregar_dados_climaticos(CAMINHO_ARQUIVO_CSV)

# 2. Calcular precipitação acumulada
if dados_base is not None:
    dados_com_acumulados = calcular_precipitacao_acumulada(dados_base)
else:
    dados_com_acumulados = None

# 3. Gerar alertas
if dados_com_acumulados is not None:
    alertas_gerados = gerar_alertas_precipitacao(dados_com_acumulados, MEUS_LIMIARES_DE_ALERTA)
else:
    alertas_gerados = None

# 4. Correlacionar alertas com imagens
if alertas_gerados is not None and not alertas_gerados.empty:
    alertas_finais_com_imagens = correlacionar_alertas_com_imagens(alertas_gerados, LISTA_NOMES_IMAGENS)

    if alertas_finais_com_imagens is not None and not alertas_finais_com_imagens.empty:
        print("\n--- RESULTADO FINAL: Amostra de Alertas Correlacionados com Imagens ---")
        pd.set_option('display.max_columns', None)
        print(alertas_finais_com_imagens.head())
        pd.reset_option('display.max_columns')

        print(f"\nTotal de alertas finais correlacionados com imagens: {len(alertas_finais_com_imagens)}")

        # Análise mais detalhada dos resultados correlacionados
        print("\n--- Análise Detalhada dos Alertas Correlacionados ---")
        for nome_imagem, grupo in alertas_finais_com_imagens.groupby('imagem_associada'):
            data_img_obj = extrair_data_de_nome_arquivo(nome_imagem)
            print(f"\nImagem: {nome_imagem} (Data: {data_img_obj})")
            print(f"  {len(grupo)} alertas associados neste dia:")
            print(grupo[['timestamp', 'estacao', 'municipio', 'nivel_alerta', 'descricao_alerta', 'valor_observado_mm']].head(10)) # Mostrar até 10 por imagem
            print("-" * 40)
    elif alertas_gerados is not None: # Se houve alertas mas nenhuma correlação
        print("\nNenhuma correlação encontrada entre os alertas gerados e as imagens fornecidas.")
    # else: # Se não houve alertas, a mensagem já foi dada por gerar_alertas_precipitacao

elif dados_com_acumulados is not None: # Se não houve alertas
     print("Nenhum alerta foi gerado a partir dos dados de chuva acumulada.")

else: # Se o carregamento inicial falhou
    print("Processamento interrompido devido à falha no carregamento dos dados iniciais.")

print("\n--- Fim do Processamento ---")